In [1]:
import cvxpy as cp
import numpy as np
import random

In [2]:
def queueGenerator(numQueue):
    listDataEquip, listDataPcard, listDataPtime, listDataPriority = [], [], [], []
    listEquip = ['4082', 'V93K']
    listPcard = ['AV44', 'FF44']

    for _ in range(numQueue):
        equip = random.choices(listEquip, weights=[0.8, 0.2])
        pcard = random.choices(listPcard, weights=[0.8, 0.2])
        # 0~100까지
        ptime = np.random.beta(1, 5, 1).item()*100
        priority = random.choices(range(0, 7), weights=[0.8, 0.01, 0.01, 0.01, 0.02, 0.05, 0.1])
        listDataEquip.append(equip)
        listDataPcard.append(pcard)
        listDataPtime.append(ptime)
        listDataPriority.append(priority)
    return listDataEquip, listDataPcard, listDataPtime, listDataPriority

numQueue = int(np.random.beta(1, 5, 1).item()*50 + 30)
listDataEquip, matrixPcard, p, priority = queueGenerator(numQueue)

In [ ]:
J = len(listDataEquip) # number of jobs
M = 14 # number of machines
st = 0.16 # release time
BIG_M = 1000
matrixEquip = np.array([[listDataEquip[j] == listDataEquip[m] for j in range(J)] for m in range(M)])
C = cp.Variable(J, nonneg=True)
z = cp.Variable((J, M), boolean=True) # 할당 여부
o = cp.Variable((J, J), boolean=True) # 순서 결정

def warm_start_heuristic(J, M, p, matrixEquip, matrixPcard, st=0.16):
    C_init = np.zeros(J)
    z_init = np.zeros((J, M))
    o_init = np.zeros((J, J))
    machine_ready = np.zeros(M)
    last_pcard = [""] * M

    # Step 1: Job 0~13 → Machine 0~13 고정 할당
    for j in range(M):
        z_init[j, j] = 1
        start_time = machine_ready[j]
        process_time = p[j]
        C_init[j] = start_time + process_time
        machine_ready[j] = C_init[j]
        last_pcard[j] = matrixPcard[j]
        # 순서정보: j보다 큰 job들에 대해 j가 먼저라고 설정
        for k in range(j+1, J):
            o_init[j, k] = 1

    # Step 2: 나머지 Job → 가능한 머신 중 가장 빨리 끝나는 곳에 할당
    for j in range(M, J):
        compatible_machines = [m for m in range(M) if matrixEquip[m][j]]
        if not compatible_machines:
            continue  # 할당 불가 → skip
        # setup time 고려한 실제 시작 가능 시간 계산
        earliest_times = [
            machine_ready[m] + (st if matrixPcard[j] != last_pcard[m] else 0)
            for m in compatible_machines
        ]
        m_best = compatible_machines[np.argmin(earliest_times)]
        start_time = earliest_times[np.argmin(earliest_times)]
        z_init[j, m_best] = 1
        C_init[j] = start_time + p[j]
        machine_ready[m_best] = C_init[j]
        last_pcard[m_best] = matrixPcard[j]

        # 순서 정보 설정
        for k in range(J):
            if k == j: continue
            if z_init[k, m_best] == 1 and C_init[k] <= start_time:
                o_init[k, j] = 1

    return z_init, o_init, C_init

z_init, o_init, C_init = warm_start_heuristic(J, M, p, matrixEquip, matrixPcard, st=0.16)
z.value = z_init
o.value = o_init
C.value = C_init

# ----- 제약 조건 정의 -----
constraints = []

# 각 Job은 정확히 하나의 Machine에 할당
for j in range(J):
    constraints.append(cp.sum(z[j, :]) == 1)
    if j < M:
        constraints.append(z[j, j] == 1)
        constraints.append(C[j] == p[j])
        for k in range(M, J):
            constraints.append(o[j, k] == 1)
    else:
        constraints.append(C[j] >= p[j])
    

    

# 장비 호환성
for j in range(M, J):
    for m in range(M):
        if not matrixEquip[m][j]:
            constraints.append(z[j, m] == 0)  
        

# 순서제약: 같은 machine에 할당된 job들만
for i in range(J):
    for j in range(J):
        if i < j:
            # constraints.append(o[i, j] + o[j, i] <= 1)
            compatible_machines = [m for m in range(M) if matrixEquip[m, i] and matrixEquip[m, j]]
            if not compatible_machines:
                continue  # skip if no compatible machine pair
            for m in compatible_machines:
                constraints.append(
                    C[j] >= C[i] + p[j] + st * (matrixPcard[i] != matrixPcard[j])
                    - BIG_M * (3 - z[i, m] - z[j, m] - o[i, j])
                )
                constraints.append(
                    C[i] >= C[j] + p[i] + st * (matrixPcard[i] != matrixPcard[j])
                    - BIG_M * (3 - z[i, m] - z[j, m] - o[j, i])
                )
                
                # 동일 machine에 2개의 job이 있을 경우 반드시 순서가 존재해야함
                # constraints.append(z[i, m] + z[j, m] <= 1 + o[i, j] + o[j, i])
                if (i >= M) and (j >= M):
                    # 우선순위 반영 (높은 priority가 먼저)
                    if priority[i] < priority[j]:
                        constraints.append(o[j, i] >= z[i, m] + z[j, m] - 1)
                    else:
                        constraints.append(o[i, j] >= z[i, m] + z[j, m] - 1)
                else:
                    constraints.append(o[i, j] >= z[i, m] + z[j, m] - 1)
                # constraints.append(o[i, j] + o[j, i] >= z[i, m] + z[j, m] - 1)

# priority_penalty_terms = []
# for i in range(J):
#     for j in range(J):
#         if i < j and priority[i] < priority[j]:
#             priority_penalty_terms.append(1 - o[j, i])  # o[j, i]가 1이 아니면 패널티 발생
                
# 우선순위 반영 (높은 priority가 먼저)
# for i in range(M, J):
#     for j in range(M, J):
#         if priority[i] > priority[j]:
#             constraints.append(o[i, j] == 1)
#         elif priority[i] < priority[j]:
#             constraints.append(o[j, i] == 1)
#         elif i < j:
#             constraints.append(o[i, j] == 1)

# ----- 목적 함수: 평균 완료 시간 최소화 -----
obj = cp.Minimize(cp.mean(C))
# alpha = 10.0  # 우선순위 위반 패널티 가중치
# obj = cp.Minimize(
#     cp.mean(C) + alpha * cp.sum(priority_penalty_terms)
# )

# ----- 문제 정의 및 풀기 -----
prob = cp.Problem(obj, constraints)
# prob.solve(solver=cp.HIGHS, verbose=True, time_limit=600)
prob.solve(solver=cp.HIGHS, verbose=True)

# ----- 결과 출력 -----
print('상태:', prob.status)
print('평균 Start Time:', prob.value)

                                     CVXPY                                     
                                     v1.6.5                                    
(CVXPY) Jun 14 04:11:50 AM: Your problem has 2596 variables, 26661 constraints, and 0 parameters.
(CVXPY) Jun 14 04:11:54 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jun 14 04:11:54 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jun 14 04:11:54 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jun 14 04:11:54 AM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jun 14 04:11:58 AM: Compiling problem (target solver=HIGHS

In [4]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

ganttData = []
for j in range(J):
    idxMachine = int(np.argmax(z.value[j], axis=0))
    
    # 직전 할당된 job과 pcard 차이가 있는지 확인인
    if (j >= M): 
        arrayZ = z[:, idxMachine].value
        idxBeforeJob = np.where(arrayZ[:j])[0][-1]
        if (matrixPcard[idxBeforeJob] != matrixPcard[j]):
            ganttData.append({
            'Job': f'Setup {j}',
            'Priority': priority[j],
            'Pcard': matrixPcard[j],
            'Start': float(C.value[j] - p[j] - st),
            'End': float(C.value[j] - p[j]),
            'Machine': f'Machine {idxMachine}'
        })
    ganttData.append({
        'Job': f'Job {j}',
        'Priority': priority[j],
        'Pcard': matrixPcard[j],
        'Start': float(C.value[j] - p[j]),
        'End': float(C.value[j]),
        'Machine': f'Machine {idxMachine}'
    })
dfGantt = pd.DataFrame(ganttData)

machine_list = sorted(dfGantt['Machine'].unique())
machine_to_y = {machine: idx for idx, machine in enumerate(machine_list)}

# Plotly 그래프 객체 초기화
fig = go.Figure()

# 각 작업/Setup을 개별 막대로 그림
for _, row in dfGantt.iterrows():
    fig.add_trace(go.Bar(
        x=[row['End'] - row['Start']],
        y=[machine_to_y[row['Machine']]],
        base=[row['Start']],
        orientation='h',
        name=row['Job'],
        hovertext=f"{row['Job']}: {row['Start']} ~ {row['End']} | {row['Priority']} | {row['Pcard']}",
        hoverinfo='text',
        # marker_color='lightcoral' if 'Setup' in row['Job'] else 'skyblue',
        showlegend=False
    ))

# y축 설정
fig.update_yaxes(
    tickvals=list(machine_to_y.values()),
    ticktext=list(machine_to_y.keys()),
    title='Machine',
    autorange='reversed'
)

# x축 수치형으로 설정
fig.update_xaxes(title='Time (unit)', type='linear')

# 기타 레이아웃
fig.update_layout(
    title='Gantt Chart (with numeric x-axis using Plotly)',
    barmode='stack',
    height=300 + 50 * len(machine_to_y)
)

fig.show()

In [44]:
z.value[:, 2]

array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0.,
       1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 0., 0., 0., 1., 0., 1., 1.])

In [45]:
o.value[16, :]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [14]:
dfGantt.head(40)

,Job,Priority,Pcard,Start,End,Machine
0,Job 0,[0],[AV44],0.000000,11.461174,Machine 0
1,Job 1,[0],[AV44],0.000000,27.463097,Machine 1
2,Job 2,[0],[AV44],0.000000,39.636360,Machine 2
3,Job 3,[0],[FF44],0.000000,15.941436,Machine 3
4,Job 4,[0],[FF44],0.000000,1.600616,Machine 4
5,Job 5,[0],[AV44],0.000000,13.714956,Machine 5
6,Job 6,[0],[AV44],0.000000,1.793713,Machine 6
7,Job 7,[6],[FF44],0.000000,30.125532,Machine 7
8,Job 8,[0],[AV44],0.000000,2.083279,Machine 8
9,Job 9,[2],[AV44],0.000000,6.471789,Machine 9
